<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [13]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""

project = hopsworks.login(project="lab1_mohamed_Emile")
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1166484
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "sweden", "city": "stockholm", "street": "stockholm-anders-lundstroms-gata", "aqicn_url": "https://api.waqi.info/feed/A61420", "latitude": 59.33, "longitude": 18.07}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [14]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
lagged_air_quality_fg = fs.get_feature_group(
    name='lagged_air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [15]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,8.0,sweden,stockholm,stockholm-anders-lundstroms-gata,2024-11-19,https://api.waqi.info/feed/A61420


In [16]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [17]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-19,0.30,0.0,4.693825,85.601379,stockholm
1,2024-11-20,1.00,0.3,20.523157,344.744812,stockholm
2,2024-11-21,-2.25,0.1,29.515608,300.004822,stockholm
3,2024-11-22,-5.00,0.0,13.905509,291.250580,stockholm
4,2024-11-23,-4.15,0.0,11.918120,295.016846,stockholm
5,2024-11-24,0.25,0.0,9.290511,144.462234,stockholm
6,2024-11-25,5.55,0.0,15.807239,210.068497,stockholm
7,2024-11-26,7.85,0.0,22.183128,221.710663,stockholm
8,2024-11-27,1.55,0.0,9.659814,296.564972,stockholm
9,2024-11-28,2.80,0.0,16.199999,233.130020,stockholm


In [18]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [19]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-19 20:30:18,333 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1166484/fs/1157187/fg/1351122


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1166484/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f4193323940>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 675006
         }
       },
       "result": {
         "observed_value": 8.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T07:30:18.000333Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsu

In [20]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-19 20:41:07,404 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1166484/fs/1157187/fg/1351046


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1166484/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f41931af550>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 674903
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T07:41:07.000404Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_betw

__Calculate the mean of air quality for the last three days and upload it to the feature store__

In [45]:
# Query the feature group for the last three rows
query = air_quality_fg.select_all()
df_last_three = query.read().sort_values(by="date", ascending=False).head(4).iloc[1:].reset_index(drop=True)
df_last_three

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.11s) 


,date,pm25,country,city,street,url
0,2024-11-18 00:00:00+00:00,2.00,sweden,stockholm,stockholm-anders-lundstroms-gata,https://api.waqi.info/feed/A61420
1,2024-11-08 00:00:00+00:00,2.20,sweden,stockholm,stockholm-anders-lundstroms-gata,https://api.waqi.info/feed/A61420
2,2024-11-07 00:00:00+00:00,0.88,sweden,stockholm,stockholm-anders-lundstroms-gata,https://api.waqi.info/feed/A61420


In [46]:
# Calculate the mean of pm25
pm25_mean = df_last_three["pm25"].mean()

# Get the first date
date = df_last_three["date"].max()

# Create the new DataFrame
df_mean_air_quality = pd.DataFrame({
    "date": [date],
    "lagged_pm25": [pm25_mean]
})

df_mean_air_quality["lagged_pm25"] = df_mean_air_quality["lagged_pm25"].astype("float64")
df_mean_air_quality

,date,lagged_pm25
0,2024-11-18 00:00:00+00:00,1.693333


In [47]:
# Insert new data
lagged_air_quality_fg.insert(df_mean_air_quality)

Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: lagged_air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1166484/jobs/named/lagged_air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f4154541ea0>, None)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
